In [14]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../../pytools')
import d2l

d2l.gpu_mem_init()

Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [15]:
def corr2d_multi_in(x, k):
    return tf.reduce_sum([d2l.corr2d(x, k) for x,k in zip(x, k)], axis=0)

In [16]:
X = tf.constant([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = tf.constant([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(f'X.shape:{X.shape}, K.shape:{K.shape}')
corr2d_multi_in(X, K)

X.shape:(2, 3, 3), K.shape:(2, 2, 2)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 56.,  72.],
       [104., 120.]], dtype=float32)>

In [17]:
def corr2d_multi_in_out(x, k):
    return tf.stack([corr2d_multi_in(x, k0) for k0 in k], axis=0)

In [18]:
K = tf.stack((K, K + 1, K + 2), axis=0)
K.shape

TensorShape([3, 2, 2, 2])

In [19]:
print(f'X.shape:{X.shape}, K.shape:{K.shape}')
corr2d_multi_in_out(X, K)

X.shape:(2, 3, 3), K.shape:(3, 2, 2, 2)


<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
array([[[ 56.,  72.],
        [104., 120.]],

       [[ 76., 100.],
        [148., 172.]],

       [[ 96., 128.],
        [192., 224.]]], dtype=float32)>

In [20]:
def corr2d_multi_in_out_1x1(x, k):
    len_input, h, w = x.shape
    len_output = k.shape[0]
    x = tf.reshape(x, (len_input, h * w))
    k = tf.reshape(k, (len_output, len_input))
    y = tf.reshape(tf.matmul(k, x),(len_output, h, w))
    return y


In [21]:
X = tf.random.normal((3, 3, 3), 0, 1)
K = tf.random.normal((2, 3, 1, 1), 0, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(tf.reduce_sum(tf.abs(Y1 - Y2))) < 1e-6